# <font color=black> Figure 3 Spinal cord func / morphometry coupling </font>
<hr style="border:1px solid black">


### Imports

In [ ]:
import sys,json
import glob, os
import pandas as pd
import numpy as np

main_dir="/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/2025_brsc_aging_project"
sys.path.append(main_dir + "/code/")

#statistics
from brsc_statistics import Statistics 
from scipy.stats import spearmanr

# Plotting
from plotting import Plotting

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load config file ------------------------------------------------------------
config_file=main_dir + '/config/analyses/brsc_sfc.json'
with open(config_file) as config_f: # the notebook should be in 'xx/notebook/' folder #config_proprio
    config = json.load(config_f) # load config file should be open first and the path inside modified

plot=Plotting(config_file,"test")
stat_func=Statistics(config=config,ana_dir="",analysis="")

metadata = pd.read_csv(config["project_dir"] + config["population_infos"], delimiter='\t')


<hr style="border:1px solid black">

### <font color=#0B7CC3> A. Load data and initiate variables
#### <font color=#0B7CC3> A.1 Matrices (similarity and FC)


In [ ]:
files={};matrix={};df_matrix_all={};df_matrix_aging={};age=[];sex=[]

#---- Initiate variables
common_ids = sorted(set(config["participants_IDs_DWI"]) & set(config["participants_IDs_MTR"]) & set(config["participants_IDs_T2s"]))
excluded_rois = ["C1_RV", "C1_RD", "C1_LV", "C1_LD"]
sub_metrics={"MTR":["MTR"],"T2s":["gm","wm"],"DWI":["FA","RD","AD","MD"]}

for modality in ["func_tp","func_fc","structural"]:
    #----- Load indiv data
    files[modality]=[];matrix[modality]=[]
    for ID_nb, ID in enumerate(common_ids):
        files[modality].append(glob.glob(config["project_dir"] + config[modality]["analysis_dir"]["spinalcord"] + config[modality]["first_level"]["global"].format(ID))[0])
        matrix[modality].append(np.array(pd.read_csv(files[modality][ID_nb],header=None)))

        if modality=="func_tp":
            age.append(metadata[metadata["participant_id"] == ID]["age"].values[0])
            sex.append(metadata[metadata["participant_id"] == ID]["sex"].values[0])
    matrix[modality] =np.array( matrix[modality]) 
    
    #---- Load group data and filter the dataframe
    df_matrix_all[modality]=pd.read_csv(glob.glob(config["project_dir"] + config[modality]["analysis_dir"]["spinalcord"] + config[modality]["second_level_matrix"])[0])
    df_matrix_all[modality]=df_matrix_all[modality][df_matrix_all[modality]["IDs"].isin(common_ids)]
    df_matrix_aging[modality]=pd.read_csv(glob.glob(config["project_dir"] + config[modality]["analysis_dir"]["spinalcord"] + config[modality]["second_level_matrix_age"])[0])
    df_matrix_aging[modality]=df_matrix_aging[modality][df_matrix_aging[modality]["IDs"].isin(common_ids)]
    
        
df_matrix_all["func_tp"] = df_matrix_all["func_tp"][~df_matrix_all["func_tp"]["seed1"].isin(excluded_rois) & ~df_matrix_all["func_tp"]["seed2"].isin(excluded_rois)].reset_index(drop=True)
df_matrix_all["func_fc"]= df_matrix_all["func_fc"][~df_matrix_all["func_fc"]["seed1"].isin(excluded_rois) & ~df_matrix_all["func_fc"]["seed2"].isin(excluded_rois)].reset_index(drop=True)
df_matrix_aging["func_tp"] = df_matrix_aging["func_tp"][~df_matrix_aging["func_tp"]["seed1"].isin(excluded_rois) & ~df_matrix_aging["func_tp"]["seed2"].isin(excluded_rois)].reset_index(drop=True)
df_matrix_aging["func_fc"]= df_matrix_aging["func_fc"][~df_matrix_aging["func_fc"]["seed1"].isin(excluded_rois) & ~df_matrix_aging["func_fc"]["seed1"].isin(excluded_rois)].reset_index(drop=True)

# ----- Grouped all matrix in the same DF
all_df=df_matrix_all["func_tp"];all_df["fc"]=df_matrix_all["func_fc"]["fcorr"];all_df["struc"]=df_matrix_all["structural"]["sim"]
all_df["rois"] = all_df["seed1"] + "_" + all_df["seed2"]
aging_df=df_matrix_aging["func_tp"];aging_df["fc"]=df_matrix_aging["func_fc"]["fcorr"];aging_df["struc"]=df_matrix_aging["structural"]["sim"]
aging_df["rois"] = aging_df["seed1"] + "_" + aging_df["seed2"]

all_df

#### <font color=#0B7CC3> A.2 Load features values (structural and functional)

In [ ]:
df_metrics_all={};df_metrics_reduced={};structural_features=[]
for modality in ["func_tp","func_fc","structural"]:
    
    #---- Load group data and filter the dataframe
    if modality != "structural":
        df_metrics_all[modality]=pd.read_csv(glob.glob(config["project_dir"] + config[modality]["analysis_dir"]["spinalcord"] + config[modality]["second_level_metrics"])[0])
        df_metrics_all[modality] = df_metrics_all[modality][df_metrics_all[modality]["IDs"].isin(common_ids)]
        
        if modality=="func_tp":
            df_metrics_all[modality] = df_metrics_all[modality][~df_metrics_all[modality]["rois"].isin(excluded_rois)].reset_index(drop=True)
            functional_features = df_metrics_all[modality].columns[np.r_[8:11,12:19,20:33]] # select feature columns
        else:
            df_metrics_all[modality] = df_metrics_all[modality][~df_metrics_all[modality]["seed1"].isin(excluded_rois) &~df_metrics_all[modality]["seed2"].isin(excluded_rois)].reset_index(drop=True)
            df_metrics_all[modality]=df_metrics_all["func_fc"].groupby(["IDs","group","age","sex","labels1","level_labels"])["fcorr"].mean().reset_index()
            df_metrics_all[modality]=df_metrics_all["func_fc"][df_metrics_all["func_fc"]["labels1"].isin(["ventral","dorsal"])].reset_index(drop=True)

    else:
        for metric in ["MTR","T2s","DWI"]:
            for sub_metric in sub_metrics[metric]:
                structural_features.append(sub_metric) # select feature columns
                df_metrics_all[modality + "_" + sub_metric]=pd.read_csv(glob.glob(config["project_dir"] + config[modality]["analysis_dir"]["spinalcord"] + config[modality]["second_level_metrics"][metric][sub_metric])[0])
                df_metrics_all[modality + "_" + sub_metric] = df_metrics_all[modality + "_" + sub_metric][df_metrics_all[modality + "_" + sub_metric]["IDs"].isin(common_ids)]
                df_metrics_all[modality + "_" + sub_metric] = df_metrics_all[modality + "_" + sub_metric][~df_metrics_all[modality + "_" + sub_metric]["rois"].isin(excluded_rois)].reset_index(drop=True)
     
                #---- Group structural metrics in a single DF    
                if metric=="MTR":
                    df_metrics_all["structural"]=df_metrics_all["structural_MTR"]
                elif metric=="T2s":
                    df_metrics_all["structural"][sub_metric]=df_metrics_all[modality + "_" + sub_metric]["T2s_vx_" + sub_metric]
                else:
                    df_metrics_all["structural"][sub_metric]=df_metrics_all[modality + "_" + sub_metric][sub_metric]
                

#----- merge structural and functional df
df_metrics_merged_VD=[]
df_metrics_merged = pd.merge(df_metrics_all["structural"], df_metrics_all["func_tp"], on=["IDs","age","sex" ,"groups","rois","ventro_dorsal","right_left","levels"], suffixes=("_struc", "_func"))
df_metrics_merged_VD=df_metrics_merged.groupby(["IDs","age","sex","ventro_dorsal","levels"])[list(structural_features) + list(functional_features)].mean().reset_index()
df_metrics_merged_VD["fc"]=df_metrics_all["func_fc"]["fcorr"]
df_metrics_merged_VD["rois"] = df_metrics_merged_VD["ventro_dorsal"] + "_" + df_metrics_merged_VD["levels"]

#--- Reduce the df by one value / participant
df_metrics_merged_reduced=[];df_metrics_merged_VD_reduced=[]
df_metrics_merged_reduced=df_metrics_merged.groupby(["rois"])[list(structural_features) + list(functional_features)].mean().reset_index()
#df_metrics_merged_VD_reduced=df_metrics_merged_VD.groupby(["ventro_dorsal","levels"])[structural_features + functional_features.tolist() + ['fc']].mean().reset_index()
df_metrics_merged_VD_reduced=df_metrics_merged_VD.groupby(["ventro_dorsal","levels"])[list(structural_features) + list(functional_features) + ['fc']].mean().reset_index()
df_metrics_merged_VD_reduced["rois"]=df_metrics_merged_VD_reduced["ventro_dorsal"] + "_" + df_metrics_merged_VD_reduced["levels"]


<hr style="border:1px solid black">

### <font color=#0B7CC3> A. Structural and functional coupling



#### <font color=#0896ae>A.1  SpiMorpho & SpiFC coupling

In [ ]:
output_dir=config["project_dir"] + '/figures/f03_sfc/revision_R1/spinalcord/'
df_SpiFC_SpiMorpho,data_SpiFC_SpiMorpho=stat_func.compute_regional_coupling(IDs=common_ids,matrix1=matrix["func_fc"][:, 4:, 4:],matrix2=matrix["structural"][:, :, :],metrics=["SpiFC","SpiMorpho"],df_out=True,metadata_df=metadata)
r_value, p_value = spearmanr(data_SpiFC_SpiMorpho['SpiFC'], data_SpiFC_SpiMorpho["SpiMorpho"])
print(r_value); print(p_value)
plot.regplots(df=data_SpiFC_SpiMorpho,x_data="SpiFC",y_data="SpiMorpho",reg_color="#000000",x_color="#04AF98",y_color="#0B7CC3",
             save=False,output_f=output_dir + "/SpiMorpho_SpiFC_coupling")

#data_SpiFC_SpiMorpho.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_3B_SpiFC_Morpho_coupling.csv", index=False)

#### <font color=#5e5f93>A.2  Morpho similarity & Temporal profile similarity coupling

In [ ]:
df_SpiDyn_SpiMorpho,data_SpiDyn_SpiMorpho=stat_func.compute_regional_coupling(IDs=common_ids,matrix1=matrix["func_tp"][:, 4:, 4:],matrix2=matrix["structural"][:, :, :],metrics=["SpiDyn","SpiMorpho"],df_out=True,metadata_df=metadata)
r_value, p_value = spearmanr(data_SpiDyn_SpiMorpho['SpiDyn'], data_SpiDyn_SpiMorpho["SpiMorpho"])
print(r_value); print(p_value)
plot.regplots(df=data_SpiDyn_SpiMorpho,x_data="SpiDyn",y_data="SpiMorpho",reg_color="#000000",x_color="#B14263",y_color="#0B7CC3",
             save=False,output_f=output_dir + "/SpiMorpho_SpiDyn_coupling")

#data_SpiDyn_SpiMorpho.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_3B_SpiDyn_Morpho_coupling.csv", index=False)

### <font color=#0B7CC3> B. Global Age-related effect


#### <font color=#0896ae>B.1  Morpho & FC coupling age-related effect

In [ ]:
roi_results_fc=[];roi_results_sim=[]

for roi in all_df["rois"].unique():
    df_roi_metrics=all_df[all_df["rois"]==roi]
    signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_sex=stat_func.regression_model(df=df_roi_metrics,y="fc",predictor="age",covariates=["sex"])
    roi_results_fc.append({"rois":roi,"signed_r2": signed_r2,"p_age": p_age,"beta_age": beta_age,"beta_sex": beta_sex,"tvalue_age": stat_age,"tvalue_sex": stat_sex})
    
    signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_sex=stat_func.regression_model(df=df_roi_metrics,y="struc",predictor="age",covariates=["sex"])
    roi_results_sim.append({"rois":roi,"signed_r2": signed_r2,"p_age": p_age,"beta_age": beta_age,"beta_sex": beta_sex,"tvalue_age": stat_age,"tvalue_sex": stat_sex})
    
    roi_results_fc_df= pd.DataFrame(roi_results_fc)
    roi_results_sim_df= pd.DataFrame(roi_results_sim)
    
    all_age_eff=pd.DataFrame({"rois":roi_results_fc_df["rois"],
            "r2_SpiFC":roi_results_fc_df["tvalue_age"],
            "r2_SpiMorpho":roi_results_sim_df["tvalue_age"]})


r_value, p_value = spearmanr(all_age_eff["r2_SpiFC"], all_age_eff["r2_SpiMorpho"])
print(r_value); print(p_value)

plot.regplots(df=all_age_eff,x_data="r2_SpiFC",y_data="r2_SpiMorpho",reg_color="#000000",x_color="#04AF98",y_color="#0B7CC3"
              ,ymin=-4, ymax=4,xmin=-3, xmax=4.5,
             save=False,output_f=output_dir + "/morpho_fc_coupling_age")

#all_age_eff.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_3C_SpiFC_Morpho_coupling_aging.csv", index=False)

In [ ]:
roi_results_fc=[];roi_results_sim=[]

for roi in all_df["rois"].unique():
    df_roi_metrics=aging_df[aging_df["rois"]==roi]
    signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_sex=stat_func.regression_model(df=df_roi_metrics,y="sim",predictor="age",covariates=["sex"])
    roi_results_fc.append({"rois":roi,"signed_r2": signed_r2,"p_age": p_age,"beta_age": beta_age,"beta_sex": beta_sex,"tvalue_age": stat_age,"tvalue_sex": stat_sex})
    
    signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_sex=stat_func.regression_model(df=df_roi_metrics,y="struc",predictor="age",covariates=["sex"])
    roi_results_sim.append({"rois":roi,"signed_r2": signed_r2,"p_age": p_age,"beta_age": beta_age,"beta_sex": beta_sex,"tvalue_age": stat_age,"tvalue_sex": stat_sex})
    
    roi_results_fc_df= pd.DataFrame(roi_results_fc)
    roi_results_sim_df= pd.DataFrame(roi_results_sim)
    
    all_age_eff=pd.DataFrame({"rois":roi_results_fc_df["rois"],
            "r2_SpiFC":roi_results_fc_df["tvalue_age"],
            "r2_SpiMorpho":roi_results_sim_df["tvalue_age"]})


r_value, p_value = spearmanr(all_age_eff["r2_SpiFC"], all_age_eff["r2_SpiMorpho"])
print(r_value); print(p_value)

plot.regplots(df=all_age_eff,x_data="r2_SpiFC",y_data="r2_SpiMorpho",reg_color="#000000",x_color="#04AF98",y_color="#0B7CC3"
              ,ymin=-4, ymax=4,xmin=-3, xmax=4.5,
             save=False,output_f=output_dir + "/morpho_SpiDyn_coupling_age")
#all_age_eff.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_3F_SpiDyn_Morpho_coupling_aging.csv", index=False)